<a href="https://colab.research.google.com/github/Farrukh-Maruf/ML_projects/blob/main/ML_Housing_USA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is for California house prediciton teller,
my try to check the all prices of houses, rooms, place and etc. 
and to try to give average price of a new house given my Real Estate Company. 

In [85]:
import numpy as np
import pandas as pd
import sklearn
import pandas as pd

In [86]:
from pandas import Series
from pandas import DataFrame

In [87]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [88]:
# lets check the URL
URL = "https://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true"
df = pd.read_csv(URL)

from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

X_train = train_set.drop("median_house_value", axis=1)
y = train_set["median_house_value"].copy()

X_num = X_train.drop("ocean_proximity", axis=1)

In [89]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


# Pipeline

In [90]:
from sklearn.base import BaseEstimator, TransformerMixin
# these columns we need to work on
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self # our function is only transformer not a estimator
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room: # add_bedrooms_per_room is optional
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]


# For columns contain for only numbers

In [91]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_pipeline = Pipeline([
          ('imputer', SimpleImputer(strategy='median')),
          ('attribs_adder', CombinedAttributesAdder(add_bedrooms_per_room = True)),
          ('std_scaler', StandardScaler())             
])

In [92]:
num_pipeline.fit_transform(housing_num)

array([[ 1.27258656, -1.3728112 ,  0.34849025, ..., -0.17491646,
         0.05137609, -0.2117846 ],
       [ 0.70916212, -0.87669601,  1.61811813, ..., -0.40283542,
        -0.11736222,  0.34218528],
       [-0.44760309, -0.46014647, -1.95271028, ...,  0.08821601,
        -0.03227969, -0.66165785],
       ...,
       [ 0.59946887, -0.75500738,  0.58654547, ..., -0.60675918,
         0.02030568,  0.99951387],
       [-1.18553953,  0.90651045, -1.07984112, ...,  0.40217517,
         0.00707608, -0.79086209],
       [-1.41489815,  0.99543676,  1.85617335, ..., -0.85144571,
        -0.08535429,  1.69520292]])

# for column contains words

In [93]:
from sklearn.compose import ColumnTransformer

num_attribs = list(X_num)
cat_attribs = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

now full pipeline is ready

In [94]:
X_prepared = full_pipeline.fit_transform(X_train)

In [95]:
X_prepared[0:5,:]

array([[ 1.27258656, -1.3728112 ,  0.34849025,  0.22256942,  0.21122752,
         0.76827628,  0.32290591, -0.326196  , -0.17491646,  0.05137609,
        -0.2117846 ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.70916212, -0.87669601,  1.61811813,  0.34029326,  0.59309419,
        -0.09890135,  0.6720272 , -0.03584338, -0.40283542, -0.11736222,
         0.34218528,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-0.44760309, -0.46014647, -1.95271028, -0.34259695, -0.49522582,
        -0.44981806, -0.43046109,  0.14470145,  0.08821601, -0.03227969,
        -0.66165785,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 1.23269811, -1.38217186,  0.58654547, -0.56148971, -0.40930582,
        -0.00743434, -0.38058662, -1.01786438, -0.60001532,  0.07750687,
         0.78303162,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-0.10855122,  0.5320839 ,  1

Linear Regression

In [96]:
from sklearn.linear_model import LinearRegression
LR_model = LinearRegression()


In [97]:
LR_model.fit(X_prepared, y)

LinearRegression()

Done of Machine Learning

In [98]:
# lets take random 5 indexes
test_data = X_train.sample(5)
test_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
19028,-122.01,38.37,16.0,3996.0,550.0,1673.0,539.0,5.7780,INLAND
4759,-118.34,34.04,37.0,1466.0,529.0,1835.0,500.0,1.7014,<1H OCEAN
5484,-118.46,33.98,32.0,2388.0,591.0,1009.0,556.0,5.2121,<1H OCEAN
10432,-117.62,33.43,27.0,3858.0,1062.0,2321.0,873.0,3.3155,NEAR OCEAN
2033,-119.76,36.73,39.0,1553.0,363.0,1449.0,341.0,1.4419,INLAND


In [99]:
test_label = y.loc[test_data.index]
test_label

19028    175700.0
4759     123200.0
5484     466700.0
10432    231000.0
2033      45500.0
Name: median_house_value, dtype: float64

In [100]:
test_data_prepared =  full_pipeline.transform(test_data)
test_data_prepared

array([[-1.21046981e+00,  1.27625668e+00, -1.00048937e+00,
         6.22646516e-01,  2.74541832e-02,  2.16835741e-01,
         1.02408251e-01,  9.96328966e-01,  8.28757161e-01,
         5.98952629e-04, -1.29704469e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 6.19413099e-01, -7.50327052e-01,  6.65897216e-01,
        -5.40796063e-01, -2.26658175e-02,  3.59313201e-01,
         3.43382980e-05, -1.14447682e+00, -1.04856211e+00,
         4.94920824e-02,  2.55179494e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 5.59580415e-01, -7.78409043e-01,  2.69138504e-01,
        -1.16806317e-01,  1.25307518e-01, -3.67145951e-01,
         1.47032777e-01,  6.99149455e-01, -4.77639911e-01,
        -1.10741874e-01,  5.97125230e-01,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 9.78409201e-01, -1.03582730e

NOw to predict data

In [101]:
predicted_data = LR_model.predict(test_data_prepared)
predicted_data

array([226292.25263656, 160112.35668011, 319199.48394559, 227499.06318144,
        50680.16816293])

Now lets check how big differences between predicted one and real one


In [102]:
pd.DataFrame({'Predicted': predicted_data, 'Real price': test_label})


,Predicted,Real price
19028,226292.252637,175700.0
4759,160112.356680,123200.0
5484,319199.483946,466700.0
10432,227499.063181,231000.0
2033,50680.168163,45500.0


Only 5 of all values are not enough to say smth exact, 
lets evaluate all

In [103]:
housing_test = test_set.drop('median_house_value', axis = 1)
housing_test

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
20046,-119.01,36.06,25.0,1505.0,NaN,1392.0,359.0,1.6812,INLAND
3024,-119.46,35.14,30.0,2943.0,NaN,1565.0,584.0,2.5313,INLAND
15663,-122.44,37.80,52.0,3830.0,NaN,1310.0,963.0,3.4801,NEAR BAY
20484,-118.72,34.28,17.0,3051.0,NaN,1705.0,495.0,5.7376,<1H OCEAN
9814,-121.93,36.62,34.0,2351.0,NaN,1063.0,428.0,3.7250,NEAR OCEAN
...,...,...,...,...,...,...,...,...,...
15362,-117.22,33.36,16.0,3165.0,482.0,1351.0,452.0,4.6050,<1H OCEAN
16623,-120.83,35.36,28.0,4323.0,886.0,1650.0,705.0,2.7266,NEAR OCEAN
18086,-122.05,37.31,25.0,4111.0,538.0,1585.0,568.0,9.2298,<1H OCEAN
2144,-119.76,36.77,36.0,2507.0,466.0,1227.0,474.0,2.7850,INLAND


In [104]:
y_test = test_set['median_house_value'].copy()
y_test

20046     47700.0
3024      45800.0
15663    500001.0
20484    218600.0
9814     278000.0
           ...   
15362    263300.0
16623    266800.0
18086    500001.0
2144      72300.0
3665     151500.0
Name: median_house_value, Length: 4128, dtype: float64

In [105]:
X_test_prepared = full_pipeline.transform(X_test)

In [106]:
y_predicted = LR_model.predict(X_test_prepared)

In [107]:
from sklearn.metrics import mean_squared_error
lin_mse = mean_squared_error(y_test, y_predicted)
# RMSE to calculate
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

72701.32600762138


Well, 
 so far, its not so bad, but its not good enough though.

Lets try to use new ways

## RandomForest

In [108]:
from sklearn.ensemble import RandomForestRegressor
RF_model = RandomForestRegressor()
RF_model.fit(X_prepared, y)

RandomForestRegressor()

In [109]:
y_predicted = RF_model.predict(X_test_prepared)
lin_mse = mean_squared_error(y_test, y_predicted)
# RMSE to calculate
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

50328.00570986636


Somehow better result then before>>>

Thank you,
check me out for more